In [1]:
import sys
import os
import gzip
import re
import matplotlib.pyplot as plt
import numpy as np
import pysam
import Bio
import pandas as pd
from Bio import SeqIO

In [2]:
curpath = os.path.abspath(os.path.curdir) # зафиксируем папку

## Класс для получения данных

In [3]:
def GetSam(curpath):
    '''Get the sam files path, make a table with start posirion, sequence, CIGAR values, flags
        For input the path with files, for input a table'''
    
    files_sam_list = [x for x in os.listdir(curpath) if x.endswith("_cart.sam")] # endswith - кончается на (можно так сделать для начала)
    path_sam_list = [os.path.join(curpath, x) for x in files_sam_list] # сделали путь ко всем sam файлам

# здесь будет цикл для перебора всех sam файлов
    data = pd.read_csv(path_sam_list[1], sep = '\t') # потом 0 заменим на цикл.
    data = pd.concat([pd.DataFrame([data.columns.values], columns=data.columns), data], ignore_index=True) # сдвигаем данные (название столбцов это часть данных)

    start = data.iloc[:, 3] # столбец из sam, где начало рида
    seq = data.iloc[:, 9] # столбец из sam c самим ридом
    cigar = data.iloc[:, 5] # столбец из sam с строкой cigar (ищем гапы)
    flag = data.iloc[:, 1] # столбец из sam с строкой cigar (ищем гапы)

    data =  start.to_frame().join(seq.to_frame()).join(cigar.to_frame()).join(flag.to_frame()) # собираем таблицу из разных столбцов
    data.columns = ['Начало рида', 'Рид', 'CIGAR','Flag'] # даем названия колоонкам
    data['Начало рида'].iloc[0] = int(data['Начало рида'].iloc[0]) # переделываем ту строку, что была названием в инт
    
    return data

def GetRef(curpath):
    '''Get the ref files path, make a list with ref nucleotides
        For input the path with files, for input a list'''
    files_ref_list = [x for x in os.listdir(curpath) if x.endswith(".fasta")] # endswith - кончается на (можно так сделать для начала)
    path_ref_list = [os.path.join(curpath, x) for x in files_ref_list] # сделали путь ко всем ref файлам
    ref_fasta= SeqIO.parse(open(path_ref_list[1]),'fasta') # потом 0 можно заменить на цикл, отрываем конкретный референс
    for i in ref_fasta: # вытаскиваем саму последовательность
        bio_ref = i.seq
    ref =[]
    for i in bio_ref: # добавляем нуклеотиды в список
        ref.append(i)
    return ref

## Класс для работы с CIGAR

In [4]:
def WorkingCigar(cigar):
    '''Changing the CIGAR from get_sam(curpath).iloc['Начало рида'] to the list of tiples 
        For input the cigar string, for output the cigar list'''
    cigar = list(cigar.strip()) #сделали список
    # для начала нужно сигар разбить на блоки и переделать цифры в инт. По идее олжно получиться int(61) str(S) int(37) str(M)...
    # Каждый символ пытаюсь перевести в инт, обработка исключений на буквы
    for i in range(len(cigar)): #пробуем каждый символ списка сделать интом
        try:
            cigar[i] = int(cigar[i])
        except ValueError:
            cigar[i] = cigar[i]
            
    # Потом объединяю цифры в одно число.
    cigar_new = []
    for i in range(len(cigar)): # если символ - строка - записываем его отдельно
        if type(cigar[i]) == str:
            cigar_new.append(cigar[i])                                    
        try:
            if type(cigar[i]) == type(cigar[i+1]) == type(cigar[i+2]) : # если три символа подряд это инт, объединяем их
                cigar_new.append(int(str(cigar[i])+str(cigar[i+1])+str(cigar[i+2])))
            if type(cigar[i]) == type(cigar[i+1]) and type(cigar[i]) != type(cigar[i+2]) and type(cigar[i-1]) ==str:  # если два символа подряд одного типа (такое может быть только с инт), объединяем их  
                cigar_new.append(int(str(cigar[i])+str(cigar[i+1])))
            if type(cigar[i+1]) == str and type(cigar[i-1]) == str: # если один инт - тоже записываем его
                cigar_new.append(cigar[i])
        except IndexError: # это нужно на случай окончания строки, list index out of range
            continue
    #print(cigar_new) # проверка правильности составления списка из cigar  
    
    # удобнее сделать сопоставление буквы числам
    numbers = cigar_new[::2] # список с числом нуклеотидов
    clipping_type = [] # список с типом совпадений
    for i in range(1, len(cigar_new), 2): 
        clipping_type.append(cigar_new[i]);
        
    #print(numbers, clipping_type) # проверка правильности списков
    cigar_list = [] # делаем словарь
    for i in range(len(numbers)):
        cigar_list.append((numbers[i],clipping_type[i]))
    
    return cigar_list

def SeqAfterCigar(working_cigar, seq):
    '''Shift nucleotides as in the IGV programm
        For input the working_cigar list (after the same function) and the sequence from get_sam(curpath).iloc['Рид'], 
        for output the nucleotide string, nucleotide indexes the same with IGV programm  '''
    nucl_number = 0 # позиция нуклеотида в последовательности
    #print(seq)
    for e in working_cigar: # для каждого кортежа в сигар:
        #print(key, value) # првоерка работы функции working_cigar
        if e[1] == 'S':
            seq = seq[:nucl_number] + 'N' * e[0] + seq[nucl_number + e[0]:] # в случае софт клиппинг пока заменяем на N
        if e[1] == 'D':
            seq = seq[:nucl_number] + ' ' * e[0] + seq[nucl_number:] # в случае делеции добавим пробел
        if e[1] == 'H':
            continue # это пока, может быть hard clipping не вырезается
        if e[1] == 'I':
            seq = seq[:nucl_number] + seq[nucl_number + e[0]:] # в случае инсерции уберем эти символы
        nucl_number += e[0] # обновим позицию нуклеотидов
    #print(key, nucl_number)
    seq = seq.replace('N', '') # а теперь уберем N, склеив последовательность
    
    return seq

def WorkingReads(curpath):
    '''Делает из всех предыдущих функций таблицу рабочих ридов'''
    new_data = [] 
    for i in range(len(GetSam(curpath))): # для всех ридов
        new_data.append((GetSam(curpath)['Начало рида'].iloc[i] - 1) * ' ' #сдвинем на начало рида из таблицы
        + SeqAfterCigar(WorkingCigar(GetSam(curpath)['CIGAR'].iloc[i]),GetSam(curpath)['Рид'].iloc[i])) # и добавим результат работы seq_after_cifar
    return new_data

## Делаем словарь для покрытия

In [5]:
def RefCover(GetRef, WorkingReads):
    '''Делает словарь покрытий по конкретному референсу и сборке.
    На вход: результат работы функции GetRef и WorkingReads, 
    на выходе список словарей с покрытием для каждого нуклеотида в референсе'''
    ref_cover_list = [] # большой список всех нуклеотидов, по всей длине референса
    for i in range(len(GetRef) + 1): # по всей длине референса создаем ячейки и счетчик для всех типов нуклеотидов
        cover_dict = {}
        cover_dict['A, нуклеотид №', i+1] = 0
        cover_dict['C, нуклеотид №', i+1] = 0
        cover_dict['G, нуклеотид №', i+1] = 0
        cover_dict['T, нуклеотид №', i+1] = 0
        ref_cover_list.append(cover_dict)
    for read in WorkingReads: # для всех ридов в сборке:
        for nucl_number, nucl in enumerate(read): # и каждого нуклеотида в риде
                # в случае совпадения с конкретным типом нуклеотида, увеличиваем счетчик
                if nucl == 'A':
                    ref_cover_list[nucl_number][('A, нуклеотид №',nucl_number + 1)]  += 1
                if nucl == 'T':
                    ref_cover_list[nucl_number][('T, нуклеотид №',nucl_number + 1)]  += 1
                if nucl == 'G':
                    ref_cover_list[nucl_number][('G, нуклеотид №',nucl_number + 1)]  += 1
                if nucl == 'C':
                    ref_cover_list[nucl_number][('C, нуклеотид №',nucl_number + 1)]  += 1
    return ref_cover_list

In [6]:
# Посчитаем все существующие замены в данной сборке:
def AllChanges(GetRef, WorkingReads):
    '''Смотрим на все существующие замены в этой сборке для этой хромосомы.
        На вход берем референс и риды сборки. На выходе получаем список списков, для каждого рида свой список.
        Внутри два элемента, номер рида и номер нуклеотида с типом замены'''
    changing = [] # список замен
    for nucl_number, nucl in enumerate(GetRef): # для всех нуклеотидов референса
        for seq_number, seq in enumerate(WorkingReads): # для каждого рида сборки
            try:
                if seq[nucl_number] != ' ': # если нуклеотид рида существует в этой позиции
                    if nucl != seq[nucl_number]: # если нуклеотид референса не равен нуклеотиду рида
                        changing.append(['Рид№ {0:.0f}'.format(seq_number), 
                                         'Замена нуклеотида {0:s} на {1:s}, Номер нуклеотида {2:.0f}'.format(nucl, seq[nucl_number], nucl_number +1)])                  
            except IndexError:
                continue
    return changing
    

In [7]:
# Посчитаем количество ридов с заменами и сколько замен в каждом риде:
def ReadWithChanges(AllChanges):
    '''Считаем количество ридов с заменами и считаем число замен в риде.
        На вход подаем список списков из функции AllChanges.
        На выход список кортежей с номером рида и числом замен в этом риде'''
    read_with_change = [] # Риды с заменами
    for e in AllChanges: # Для каждого элемента из общего файла существующих замен:
        read_with_change.append(e[0]) # Добавляем сюда номер рида

    change = [] 
    for read in read_with_change:
        change.append(('В {0:s}'.format(read), read_with_change.count(read))) # Считаем сколько раз этот рид встретился в наборе всех замен
    read_with_change = list(set(change)) # Берем сет от этого набора:
    return read_with_change
    

In [23]:
# Посчитаем по позиции нуклеотида сколько каких там замен
def NuclWithChange(AllChanges):
    '''Считаем количество замен в каждом конретном нуклеотиде.
        На вход подаем список списков из функции AllChanges.
        На выход список кортежей с номером нуклеотида и типом замены и число подобых замен'''
    nucleotide_with_change = []
    for e in AllChanges:
        nucleotide_with_change.append(e[-1])

    change = []
    for read in nucleotide_with_change:
        change.append(('В {0:s}'.format(read), nucleotide_with_change.count(read)))

    nucleotide_with_change = list(set(change))
    return sorted(nucleotide_with_change, key=lambda a: a[0][-3:])


In [24]:
NuclWithChange(AllChanges(GetRef(curpath), WorkingReads(curpath)))

[('В Замена нуклеотида C на G, Номер нуклеотида 10', 28),
 ('В Замена нуклеотида T на G, Номер нуклеотида 11', 29),
 ('В Замена нуклеотида C на T, Номер нуклеотида 13', 29),
 ('В Замена нуклеотида C на G, Номер нуклеотида 14', 30),
 ('В Замена нуклеотида C на A, Номер нуклеотида 16', 31),
 ('В Замена нуклеотида C на T, Номер нуклеотида 19', 31),
 ('В Замена нуклеотида A на G, Номер нуклеотида 20', 17),
 ('В Замена нуклеотида G на T, Номер нуклеотида 21', 31),
 ('В Замена нуклеотида C на T, Номер нуклеотида 22', 32),
 ('В Замена нуклеотида G на T, Номер нуклеотида 24', 33),
 ('В Замена нуклеотида C на G, Номер нуклеотида 25', 33),
 ('В Замена нуклеотида C на T, Номер нуклеотида 26', 16),
 ('В Замена нуклеотида A на G, Номер нуклеотида 27', 33),
 ('В Замена нуклеотида T на A, Номер нуклеотида 28', 33),
 ('В Замена нуклеотида T на A, Номер нуклеотида 29', 33),
 ('В Замена нуклеотида G на T, Номер нуклеотида 30', 33),
 ('В Замена нуклеотида G на A, Номер нуклеотида 31', 33),
 ('В Замена ну